Добро пожаловать в Jupyter Notebook, веб-интерфейс интерактивного интерпретатора языка Python.

Эта программа выполняет моделирование макромолекулы как цепи с фиксированными валентными углами. Программа, исходный код которой вводится здесь, исполняется на удаленном сервере, а результаты выполнения отображаются на этой странице.

На первом шаге мы подготовим интерпретатор для выполнения моделирования: импортируем нужные модули, определим функции и константы.

Установите курсор в ячейку с кодом ниже, щелкнув по ней, и нажмите кнопку `Run` на панели инструментов сверху, чтобы исполнить шаг.

In [ ]:
%matplotlib inline

import math
from math import sin, cos, tan, asin, acos, atan2, pi
import random

import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt


# Число мономерных звеньев / сегментов полимерной цепи
SEGMENT_COUNT = 200

# Валентный угол
ANGLE_DEG = 140


def normalize(v):
    norm=np.linalg.norm(v, ord=1)
    if norm==0:
        norm=np.finfo(v.dtype).eps
    return v/norm

LENGTH = 1
angle = math.radians(ANGLE_DEG)
i_vector = np.array([1, 0, 0])

assert angle > pi / 2
assert angle < pi

r = LENGTH
theta = pi - angle

def calc():
    points = [np.array([0, 0, 0]), np.array([0.1, 0.1, math.sqrt(0.98)])]
    for index in range(SEGMENT_COUNT):
        prev_point = points[index]
        curr_point = points[index + 1]
        prev_vector = curr_point - prev_point
        circle_center = curr_point + prev_vector * cos(pi - angle)
        circle_radius = LENGTH * sin(angle)

        u = np.cross(prev_vector, i_vector)
        u = u / np.linalg.norm(u, ord=1)
        v = normalize(np.cross(prev_vector, u))
        t = random.uniform(0, 2 * pi)
        next_point = circle_center + circle_radius * cos(t) * u + circle_radius * sin(t) * v
        points.append(next_point)

    squared_dist = np.sum(points[0]**2 + points[-1]**2, axis=0)
    distance = np.sqrt(squared_dist)
    return points, distance

def draw():
    points, distance = calc()
    fig = plt.figure(figsize=[12, 7], dpi=80)
    ax = fig.add_subplot(111, projection='3d')

    for idx, point in enumerate(points):
        ax.scatter(*point, c='b', marker='o')
        if idx != 0:
            ax.plot([points[idx - 1][0], points[idx][0]],
                    [points[idx - 1][1], points[idx][1]],
                    zs=[points[idx - 1][2], points[idx][2]],
                    color='blue')

    min_x = min(a[0] for a in points)
    min_y = min(a[1] for a in points)
    min_z = min(a[2] for a in points)
    max_x = max(a[0] for a in points)
    max_y = max(a[1] for a in points)
    max_z = max(a[2] for a in points)
    avg_x = np.mean([a[0] for a in points])
    avg_y = np.mean([a[1] for a in points])
    avg_z = np.mean([a[2] for a in points])
    max_scale_length = max([max_x - min_x, max_y - min_y, max_z - min_z])
    ax.set_xlim3d([avg_x - 0.4 * max_scale_length, avg_x + 0.4 * max_scale_length])
    ax.set_ylim3d([avg_y - 0.4 * max_scale_length, avg_y + 0.4 * max_scale_length])
    ax.set_zlim3d([avg_z - 0.4 * max_scale_length, avg_z + 0.4 * max_scale_length])
    
    return distance
  
distances = []

Отлично! Теперь можно переходить непосредственно к моделированию — нажмите `Run`, установив курсор в ячейку ниже, и будет сгенерирована и отображена случайная макромолекула согласно созданной модели.

In [ ]:
distance = draw()
distances.append(distance)
print('R = {}'.format(distance))

Мы смоделировали одну макромолекулу. Полученное расстояние между концами цепи, характеризующее её гибкость, выведено над изображением.
Попробуйте ещё несколько раз выполнить последний шаг, чтобы сгенерировать несколько разных макромолекул.

После этого давайте посмотрим на распределение (гистограмму) расстояния между концами цепи — для этого запустите следующий шаг.

In [ ]:
plt.hist(distances, bins=10)

Если вы запускали моделирование не слишком много раз, скорее всего, получившаяся гистограмма выглядит не очень понятно: слишком велика роль случайности в полученном распределении. Давайте попробуем сгенерировать 1000 макромолекул и выведем получившееся распределение (это займет несколько десятков секунд):

In [ ]:
distances = []
for _ in range(1000):
    _, distance = calc()
    distances.append(distance)
plt.hist(distances, bins=20)

Совсем другое дело! Видно, что хотя каждая макромолекула генерируется случайно, распределение большого числа макромолекул по размерам (расстоянию между концами цепи) подчиняется определенному закону. Больше о нём вы можете узнать в учебнике "Высокомолекулярные соединения" под ред. А. Б. Зезина на с. 39.

Теперь вы можете изменить длину (степень полимеризации) макромолекулы или валентный угол и посмотреть, как при этом будет изменяться вид и расстояние между концами макромолекулы. Для этого измените значения переменных `SEGMENT_COUNT` (число мономерных звеньев / сегментов) и `ANGLE_DEG` (валентный угол) в начале кода первого шага и заново выполните первый шаг и последующий.


*© А. А. Коригодский*